# 단위 테스트 

## 테스트 스킵

In [9]:
!cat ./tests/test_skip.py

import pytest

try:
    import mylib
except:
    mylib = None
    
@pytest.mark.skip("Do not run this")
def test_fail():
    assert False
    
@pytest.mark.skipif(mylib is None, reason='mylib is not available')
def test_mylib():
    assert mylib.foobar() == 42
    
def test_skip_at_runtime():
    if True:
        pytest.skip('Skp this')
        
        

In [8]:
$ pytest -v ./tests/test_skip.py
================================================================================= test session starts =================================================================================
collected 3 items                                                                                                                                                                     

tests/test_skip.py::test_fail SKIPPED (Do not run this)                                                                                                                         [ 33%]
tests/test_skip.py::test_mylib SKIPPED (mylib is not available)                                                                                                                 [ 66%]
tests/test_skip.py::test_skip_at_runtime SKIPPED (Skp this)  

## -k 로 특정 테스트만 실행

In [ ]:
$ pytest -v ./tests/test_skip.py -k test_fail

## 키워드로 마킹하여 필터링 

In [10]:
!cat ./tests/test_marking.py

import pytest
 
@pytest.mark.dicttest("This is dicttest")
def test_dicttest_1():
    assert True

@pytest.mark.dicttest("This is dicttest")
def test_dicttest_2():
    assert True

@pytest.mark.listtest("This is listtest")
def test_listtest1():
    assert True

@pytest.mark.listtest("This is listtest")
def test_listtest2():
    assert True

def test_somethingelse():
    assert False

In [ ]:
$ pytest -v ./tests/test_marking.py -m dicttest
================================================================================= test session starts =================================================================================
collected 5 items / 3 deselected / 2 selected                                                                                                                                         

tests/test_marking.py::test_dicttest_1 PASSED                                                                                                                                   [ 50%]
tests/test_marking.py::test_dicttest_2 PASSED  

## 병렬 테스트

In [ ]:
!pip install pytest-xdist

In [ ]:
$ pytest -v ./tests/test_marking.py -m dicttest -n 2
================================================================================= test session starts =================================================================================
plugins: typeguard-2.10.0, xdist-2.3.0, forked-1.3.0, requests-mock-1.8.0
[gw0] Python 3.6.12 |Anaconda, Inc.| (default, Sep  8 2020, 23:10:56)  -- [GCC 7.3.0]
[gw1] Python 3.6.12 |Anaconda, Inc.| (default, Sep  8 2020, 23:10:56)  -- [GCC 7.3.0]
gw0 [2] / gw1 [2]
scheduling tests via LoadScheduling

tests/test_marking.py::test_dicttest_2 
tests/test_marking.py::test_dicttest_1 
[gw1] [ 50%] PASSED tests/test_marking.py::test_dicttest_2 
[gw0] [100%] PASSED tests/test_marking.py::test_dicttest_1 


## fixture autouse 사용

In [11]:
!cat ./tests/test_fixture.py

import pytest
import os
 
@pytest.fixture(autouse=True)
def change_user_env():
    # before
    curuser = os.environ.get("USER")
    os.environ["USER"] = "foobar"
    yield
    
    # after
    os.environ["USER"] = curuser


def test_somethingelse():
    assert os.getenv("USER") == "foobar"

## 매개변수화된 test

### 매개변수화된 단위 테스트

In [1]:
!cat tests/test_parameterized.py

import pytest


@pytest.mark.parametrize("test_input,expected", [("3+5", 8), ("2+4", 6), ("6*9", 42)])
def test_eval(test_input, expected):
    assert eval(test_input) == expected


In [ ]:
$>pytest --collect-only ./tests/test_parameterized.py 
==================================================================================================================== test session starts =====================================================================================================================
collected 3 items                                                                                                                                                                                                                                            

<Module tests/test_parameterized.py>
  <Function test_eval[3+5-8]>
  <Function test_eval[2+4-6]>
  <Function test_eval[6*9-42]>


### 매개변수화된 fixture

In [3]:
!cat tests/test_param_fixture.py

import pytest
import os
 
@pytest.fixture(params=["mysql", "maria"])
def database(request):
    yield request.param
    
def test_somethingElse(database):
    assert 0 # for demo purpose


In [ ]:
$ pytest  --collect-only ./tests/test_param_fixture.py 
==================================================================================================================== test session starts =====================================================================================================================
collected 2 items                                                                                                                                                                                                                                            

<Module tests/test_param_fixture.py>
  <Function test_somethingElse[mysql]>
  <Function test_somethingElse[maria]>

## 모의 객체를 이용한 제어된 테스트하기 

https://docs.python.org/ko/3/library/unittest.mock-examples.html

In [8]:
from unittest import mock
m = mock.Mock()
m.some_method.return_value = 42
m.some_method()

42

In [9]:
def print_hello():
    print("hello world")
    return 43

m.some_method2.side_effect = print_hello
m.some_method2()

hello world


43

In [10]:
m.some_method2.call_count

1

### 호출 확인

In [11]:
from unittest import mock
m = mock.Mock()
m.some_method('foo', 'bar')

<Mock name='mock.some_method()' id='139719259290032'>

In [12]:
m.some_method.assert_called_once_with('foo', 'bar')

In [13]:
m.some_method.assert_called_once_with('foo', 'bazzz')

AssertionError: Expected call: some_method('foo', 'bazzz')
Actual call: some_method('foo', 'bar')

### 패치하기

In [14]:
from unittest import mock
import os
def fake_os_unlink(path):
    raise IOError("Testing")
    
with mock.patch('os.unlink', fake_os_unlink):
    os.unlink('foobar')

OSError: Testing

In [ ]:
from unittest import mock
import requests

def get_fake_get(status_code, content):
    m = mock.Mock()
    m.status_code = status_code
    m.content = content

    def fake_get(url):
        return m

    return fake_get

@mock.patch('requests.get', get_fake_get(200, 'foo'))
def test_foo():
    r = requests.get('http://foo')
    assert r.content == 'foo'

@mock.patch('requests.get', get_fake_get(200, 'bar'))
def test_foo():
    r = requests.get('http://bar')
    assert r.content == 'bar'